In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
column_names=['sentiment','id','date','flag','user','text']

In [4]:
dataset=pd.read_csv("/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv",names=column_names,encoding='ISO-8859-1')

In [5]:
dataset.head()

,sentiment,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
tweets=dataset.copy()

In [7]:
tweets=tweets[['text','sentiment']]

In [8]:
tweets.head()

,text,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [9]:
tweets.sentiment=tweets.sentiment.replace(4,1)

In [10]:
tweets=tweets.loc[785002:815001,:]

In [11]:
tweets.shape

(30000, 2)

In [12]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
ps=PorterStemmer()

In [14]:
default_stopwords = set(stopwords.words('english'))
negation_words = {"not", "no", "nor", "don't", "didn't", "won't", "wouldn't", "can't", "cannot", "couldn't", "shouldn't", "wasn't", "weren't", "isn't", "ain't", "doesn't", "haven't", "hadn't"}
custom_stopwords = default_stopwords - negation_words

In [15]:
def clean(text):
     text_cleaned=re.sub('[^a-zA-Z]',' ',text).lower().split()
     text_cleaned = [ps.stem(word) for word in text_cleaned if word not in custom_stopwords]
     return ' '.join(text_cleaned)

In [16]:
tweets['text']=tweets['text'].apply(clean)

In [17]:
tweets.head()

,text,sentiment
785002,seesmic done auto account select mess,0
785003,bright sunni day wish could play outdoor alon,0
785004,willclarkfan ooh england franc itali amaz wish...,0
785005,nice shame sinc,0
785006,cook school photo line im guy left blog beard ...,0


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=50000,stop_words='english')

In [19]:
vector=cv.fit_transform(tweets['text']).toarray()

In [20]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
X=vector
y=tweets['sentiment']

In [22]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [24]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2187  799]
 [ 631 2383]]


0.7616666666666667

In [26]:
X_train.shape

(24000, 32043)

In [28]:
import pickle

In [29]:
filename='model.sav'
pickle.dump(lr,open(filename,'wb'))

In [30]:
#loading the saved model
loaded_model=pickle.load(open('/kaggle/working/model.sav','rb'))

In [50]:
sentence="google is a rising company"

In [51]:
cleaned_sentence = clean(sentence)
transformed_sentence = cv.transform([cleaned_sentence])
dense_transformed_sentence = transformed_sentence.toarray()

print("Shape of dense transformed sentence:", dense_transformed_sentence.shape)
reshaped_input = dense_transformed_sentence.reshape(1, -1)

prediction = loaded_model.predict(reshaped_input)
if(prediction==0):
    print("Negative")
else:
    print("positive")

Shape of dense transformed sentence: (1, 32043)
positive
